# NLP Topic Modelling on PureGym Reviews

**Author:** Matt Cocker  
**Date:** [July 2025]  
**Project Type:** NLP | Topic Modelling | Customer Insight

---

## About this Notebook

This notebook applies topic modelling and emotion analysis to customer reviews from PureGym to uncover key drivers of member behaviour and sentiment. It supports business insight generation by identifying dominant themes and emotional tones in user feedback.

The notebook uses traditional LDA for topic modelling, which will be compared to a separate notebook. Preprocessing steps are performed in the separate notebook, and processed data is loaded via `.pkl` files for efficiency.

**Key Tasks:**
- Load cleaned review data
- Apply topic modelling
- Visualise topic distributions

*Note: The original dataset used in this notebook is under NDA and not accessible.  Any data loading or Drive mounting steps have been replaced with placeholders.*

# Imports

We will start by force reinstalling versions of the necessary libraries which are compatible with Gensim.

In [ ]:
!pip install -q --force-reinstall --no-cache-dir numpy==1.24.4 gensim==4.3.2 pyLDAvis==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 155.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 153.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


Now we'll import the necessary libraries.

In [ ]:
# --- Standard libraries
import os
import pickle

# --- Analysis libraries
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models

# --- Data & NLTK setup
# from google.colab import drive
# drive.mount('/content/drive')
# Drive access removed for this public version – data loading uses placeholder below

nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

# Importing Data

We will import our list of negative reviews from the merged datasets in the part 1 notebook using pickle.

In [ ]:
# ⚠️ NDA Notice:
# The original data used in this project is proprietary and cannot be shared.
# Below is an example placeholder to illustrate where data would be loaded.

# with open("data/merged_neg_reviews.pkl", "rb") as f:
#     merged_neg_reviews = pickle.load(f)

print("Data loading step skipped due to confidentiality.")


Merged Dataset of Negative Reviews:
[['many', 'students', 'two', 'local', 'colleges', 'go', 'leave', 'rubbish', 'changing', 'rooms', 'sit', 'like', 'canteen', 'going', 'years', 'cancel', 'membership', 'go', 'group', 'disgusting', 'students', 'hanging', 'around', 'machines', 'messing', 'around', 'like', 'school', 'crowded', 'ceo', 'supports', 'genocide', 'civilians', 'israel', 'disgusting', 'people'], ['current', 'member', 'quite', 'dirty', 'often', 'theres', 'no', 'soap', 'bathroom', 'zero', 'airflow', 'like', 'sauna', 'also', 'often', 'overcrowded', 'anytime', 'pm', 'good', 'thing', 'location', 'bring', 'buddy', 'thing'], ['way', 'hot', 'even', 'workout', 'no', 'windows', 'open', 'ac', 'barely', 'works', 'staff', 'no', 'near', 'friendly', 'always', 'rude', 'especially', 'men', 'clients', 'mean', 'work'], ['no', 'access', 'wc', 'empty', 'no', 'assistance', 'gain', 'access', 'fault', 'forgot', 'pin', 'didnt', 'see', 'stay', 'enable', 'assistance'], ['year', 'finally', 'leaving', 'gutte

# Preprocessing

Here we'll perform the preprocessing steps - we've already removed punctuation, numbers and stopwords in previous preprocessing, as well as making the text lower case and tokenizing. As such, we just need to lemmatise the cleaned comments.

In [ ]:
# Lemmatise the cleaned reviews - we will use the negative reviews from the Top 30 Locations

lemmatizer = WordNetLemmatizer()

# Set up the lemmatiser
def lemmatise_text(text):
    lemmatised_tokens = [lemmatizer.lemmatize(token) for token in text]
    return lemmatised_tokens

neg_merged_reviews_lemma = [lemmatise_text(tokens) for tokens in merged_neg_reviews]

In [ ]:
# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(neg_merged_reviews_lemma)

# Filter out words that occur in fewer than 2 documents or in more than 50% of the documents.
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Create corpus (bag-of-words)
corpus = [dictionary.doc2bow(text) for text in neg_merged_reviews_lemma]

# Modelling

Now we'll build the LDA model, setting it to generate 10 topics.

In [ ]:
# Instantiate the LDA model with num_topics = 10

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=10,
    random_state=19,
    passes=20,
    alpha='auto',   # Set 'auto' instead of default to let model learn document-topic density from the data
    per_word_topics=True
)

# Visualisation

Now we'll visualise the output using pyLDAvis

In [ ]:
# Prepare visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# Display it
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.141557 -0.025222       1        1  21.997490
6     -0.065194  0.007761       2        1  19.984773
9      0.068632 -0.004055       3        1  14.752350
8     -0.052146 -0.011149       4        1  11.502328
7      0.167827 -0.063288       5        1   9.789965
5      0.052008 -0.011565       6        1   5.925035
3     -0.040146 -0.033189       7        1   5.539173
4     -0.161878 -0.130906       8        1   4.452562
1     -0.094006  0.038829       9        1   3.564512
0     -0.016655  0.232783      10        1   2.491812, topic_info=          Term         Freq        Total Category  logprob  loglift
774     shower   652.000000   652.000000  Default  30.0000  30.0000
191      class   711.000000   711.000000  Default  29.0000  29.0000
670    parking   235.000000   235.000000  Default  28.0000  28.0000
40          no  1652.000000  1652.000000  Default  27.0000  27.0000
21        room   546.000000   546.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
59         men    10.601706   153.755293  Topic10  -5.6432   1.0178
40          no    12.561838  1652.447305  Topic10  -5.4736  -1.1872
334        gym    11.364977   396.744859  Topic10  -5.5737   0.1394
90   equipment    11.759117  1304.808456  Topic10  -5.5396  -1.0170
20      people    10.554148   834.910146  Topic10  -5.6477  -0.6786

[790 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2208     10  0.885384       abbey
1350      1  0.071206  absolutely
1350      2  0.051786  absolutely
1350      3  0.356031  absolutely
1350      4  0.090626  absolutely
...     ...       ...         ...
28       10  0.002345        year
1803      4  0.818960      yellow
1817      1  0.053934        yoga
1817      2  0.916886        yoga
4672      9  0.929959        york

[2158 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 10, 9, 8, 6, 4, 5, 2, 1])

The low number of topics set results in some generic topics where the theme isn't clear. Moreover, the distances are sometimes odd - Topic 5 on cleanliness overlaps with Topic 1 on equipment maintenance, but is far away from Topic 8 which has the theme of cold showers. Topic 8 is also odd, with cold showers being grouped together with parking.

A more specific subset of negative reviews, such as angry reviews, may help to make more distinct clusters with clearer themes. More fine-tuning of the LDA model may also help to generate a better visualisation.